# Using LangChain on Google Cloud

## Overview
This tutorial is designed to give you the basics of using langchain to work with Large Language Models (LLMs) for document summarization and basic chat bot functionality. You could take what we have here to build a front end application using something like streamlit, or other further iterations.

## Learning Objectives
+ Learn the basics of LangChain
+ Learn how to use the Google Cloud tools from LangChain
+ Learn how to deploy and interact with LLMs
+ Learn how to use vector stores

## Prerequisites
+ You need access to Vertex AI 

## Get Started

### Install packages

In [ ]:
!pip install -U google-cloud-aiplatform langchain langchain-community langchain-google-vertexai pypdf faiss-cpu --user

### Import libraries

In [ ]:
import bs4
from langchain_google_vertexai import ChatVertexAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.schema.prompt_template import format_document
from langchain_text_splitters import RecursiveCharacterTextSplitter

! export USER_AGENT="my-langchain-script/1.0"

### Summarize a scientific article using an LLM

In [ ]:
loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/37883540/")
docs = loader.load()


In [ ]:
llm = ChatVertexAI(model_name="gemini-1.5-flash-002")
print('the LLM and default params are : ', llm)

chain = load_summarize_chain(llm, chain_type="stuff")

print('\n''the LLM chain used is ''\n', chain)


In [ ]:
print('the summary of the document in a single paragraph is: ')

print(chain.invoke(docs))


**Now try using [a different LLM](https://python.langchain.com/docs/integrations/llms/) and see if you can get the code to run!**

### Ask a general question to an LLM, without the context of a specific source

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [ ]:
chain = prompt | llm

In [ ]:
question = "What evidence do we have for chimpanzees going through menopause?"

print(chain.invoke({"question": question}))

### Build a simple Chat Bot to query specific content

### Load your PDF file

Read more about document loaders from langchain [here](https://python.langchain.com/docs/how_to/document_loader_pdf/). Note that we are both loading, and splitting our document. You can read more about the default document chunking/splitting procedures [here](https://python.langchain.com/docs/concepts/text_splitters/).

In [ ]:
!wget --user-agent "Chrome" https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10954554/pdf/41586_2024_Article_7159.pdf

In [ ]:
loader = PyPDFLoader("41586_2024_Article_7159.pdf")
pages = loader.load_and_split()

In [ ]:
# you could also load from the web url
# loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/37883540/")
# docs = loader.load()

In [ ]:
pages[0]

### Create a vector store
One of the usual methods for organizing and searching through unstructured data is to convert it into embedded vectors, which are compact (numerical) representations. These vectors are stored, and when you want to find something similar, you turn your query into an embedded vector as well. A "vector store" then manages the stored data and helps you find the most similar vectors to your query. Read more about vector stores in langchain [here](https://python.langchain.com/docs/how_to/#vector-stores). Here we are going to use a very meta technique using the Facebook AI Similarity Search (FAISS) library. You can explore the various vector store options [here](https://python.langchain.com/docs/integrations/vectorstores/). Here we are using embeddings to downselect the total information we want to feed to the LLM downstream. As token limits go up, we will eventually be able to feed a whole document to the LLM, but for now, you will usually need to use this method to downsample. If your document is small enough, just push it directly to the LLM. Also, use embeddings for when you want to query over many documents (1000's). 

In [ ]:
# index the document using FAISS
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")
faiss_index = FAISS.from_documents(pages, embeddings)

Define the user query, which will also be converted to embeddings

In [ ]:
query = 'What evidence is there that toothed whales go through menopause'

In [ ]:
pages = faiss_index.similarity_search(query, k=5)
pages[0]

Now we have summaries of our query based on the article. Now we need to pass the summaries to our LLM and generate a single summary. 

In [ ]:
doc_prompt = PromptTemplate.from_template("{page_content}")

chain = (
    {
        "content": lambda pages: "\n\n".join(
            format_document(page, doc_prompt) for page in pages
        )
    }
    | PromptTemplate.from_template("Summarize the following content in around 200 words:\n\n{content}")
    | ChatVertexAI(model="gemini-1.5-flash-002")
    | StrOutputParser()
)

In [ ]:
print(chain.invoke(pages))

Here are a few example prompts, try runnning them in the template and chain below

In [ ]:
prompt_str = "Instructions: You need to summarize text from several documents. \
                                   Be professional, factual, and succinct in the response. \
                                   Your answer is ONLY based on information in the documents above. \
                                   If you can not answer the question, answer \
                                   I am sorry, I am unable to answer the question based on the information provided \
                                   ONLY use information that is based on the documents. \
                                   \
                                   Document number: \
                                   Documents: {page_content}"

In [ ]:
doc_prompt = PromptTemplate.from_template("{page_content}")

chain = (
    {
        "page_content": lambda pages: "\n\n".join(
            format_document(page, doc_prompt) for page in pages
        )
    }
    | PromptTemplate.from_template(prompt_str) 
    | ChatVertexAI(model="gemini-1.5-flash-002")
    | StrOutputParser()
)

In [ ]:
print(chain.invoke(pages))

### Deploy a local Model
If you want to avoid sending data over the internet, you can deploy a model to an endpoint following [these instructions](https://cloud.google.com/vertex-ai/docs/general/deployment).

In [ ]:
#model garden
#https://cloud.google.com/vertex-ai/docs/general/deployment#what_happens_when_you_deploy_a_model
from langchain_google_vertexai import VertexAIModelGarden

In [ ]:
llm = VertexAIModelGarden(
    project="YOUR PROJECT ID",
    endpoint_id="YOUR ENDPOINT ID"
)

In [ ]:
llm = VertexAIModelGarden(
    project="YOUR PROJECT ID",
    endpoint_id="YOUR ENDPOINT ID"
)

In [ ]:
print(llm.invoke("What are the greatest questions left to answer in biomedical research?"))

You can repeat any of the methods shown above, but using the locally deployed LLM.

### Generate Code

In [ ]:
llm = ChatVertexAI(model_name="gemini-2.0-flash-001", max_output_tokens=1000, temperature=0.3)

In [ ]:
question = "Write a python function that checks if a string is a valid email address"

In [ ]:
print(llm.invoke(question))

In [ ]:
question = "Write a Nextflow module from nf-core to run bwa"

In [ ]:
print(llm.invoke(question))

In [ ]:
question = "Write a Snakemake module from nf-core to run bwa"

In [ ]:
print(llm.invoke(question))

## Conclusions
+ You learned how to work with LLMs in Vertex AI using LangChain
+ You learned how to use vector stores and document loaders

## Clean Up

Make sure to stop your VM